# Project 4: Part2

## What factors distinguish job category?
### Specifically, what components of a job posting distinguish data scientists from other data jobs?
#### I will have to add a column indicating 1 if data scientist, and 0 if not. Then try and find distinguishing features.

In [11]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

In [19]:
data = pd.read_csv('/Users/lohyenwei/Desktop/data_science/Projects/Project4/Project4_uploads/yen/clean_dataP4.csv')

In [20]:
data

,Unnamed: 0,title,salary,location,advertiser,industry,description,cleaned_salary,number_salary,final_salary,sal_cat,lead_count
0,0,Project Coordinator - Data entry,b'Up to $42ph plus Super',Sydney,Preacta Recruitment,Administration & Office Support,"b'<div class=""videoembed"" id=""FlashContent""></...",42,42,82992,high,793
1,1,Data Engineer / Data Scientist,b'Start up culture',Sydney,Motion Recruitment,Information & Communication Technology,"b""\n<p>Australian start up success story requi...",0,0,0,0.0,793
2,2,Data Analyst,0,Sydney,Westpac Group,Banking & Financial Services,b'<span>The Customer Data Analyst will support...,0,0,0,0.0,793
3,3,Media Data Analyst,0,Sydney,Big Wave Digital,Science & Technology,b'\n<div>\n<p>Are you looking to join massive ...,0,0,0,0.0,793
4,4,NaN,b'Attractive remuneration',Sydney,IPA,Information & Communication Technology,b'<span><strong>Surveyor - Telecommunications<...,0,0,0,0.0,793
5,5,NaN,"b'$90,000 - $100,000 plus Super + Bonus'",Sydney,Finite IT Recruitment Solutions,Information & Communication Technology,"b'\n<p><font face=""Calibri"" size=""2"">Data Ware...",90000100000,90000100000,95000,high,793
6,6,Data Steward - SAP / MDM,"b'$70,000 to $80,000 + super'",Sydney,Perigon Group Pty Limited,Information & Communication Technology,"b""Data Steward - SAP / MDM<br/>\n<br/>\r\nWe a...",7000080000,7000080000,75000,good,793
7,7,Data Engineer,0,Sydney,IAG,Information & Communication Technology,"b""\n<ul>\n<li>One of Australia's leading gener...",0,0,0,0.0,793
8,8,Data Scientist,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0,0,0.0,793
9,9,Data Analyst,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0,0,0.0,793


In [21]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop('cleaned_salary',axis=1,inplace=True)
data.drop('number_salary',axis=1,inplace=True)
data.drop('lead_count',axis=1,inplace=True)

In [22]:
data

,title,salary,location,advertiser,industry,description,final_salary,sal_cat
0,Project Coordinator - Data entry,b'Up to $42ph plus Super',Sydney,Preacta Recruitment,Administration & Office Support,"b'<div class=""videoembed"" id=""FlashContent""></...",82992,high
1,Data Engineer / Data Scientist,b'Start up culture',Sydney,Motion Recruitment,Information & Communication Technology,"b""\n<p>Australian start up success story requi...",0,0.0
2,Data Analyst,0,Sydney,Westpac Group,Banking & Financial Services,b'<span>The Customer Data Analyst will support...,0,0.0
3,Media Data Analyst,0,Sydney,Big Wave Digital,Science & Technology,b'\n<div>\n<p>Are you looking to join massive ...,0,0.0
4,NaN,b'Attractive remuneration',Sydney,IPA,Information & Communication Technology,b'<span><strong>Surveyor - Telecommunications<...,0,0.0
5,NaN,"b'$90,000 - $100,000 plus Super + Bonus'",Sydney,Finite IT Recruitment Solutions,Information & Communication Technology,"b'\n<p><font face=""Calibri"" size=""2"">Data Ware...",95000,high
6,Data Steward - SAP / MDM,"b'$70,000 to $80,000 + super'",Sydney,Perigon Group Pty Limited,Information & Communication Technology,"b""Data Steward - SAP / MDM<br/>\n<br/>\r\nWe a...",75000,good
7,Data Engineer,0,Sydney,IAG,Information & Communication Technology,"b""\n<ul>\n<li>One of Australia's leading gener...",0,0.0
8,Data Scientist,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0.0
9,Data Analyst,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0.0


In [23]:
# check for nans
data.isnull().sum()

title           144
salary            0
location          0
advertiser        0
industry          0
description      23
final_salary      0
sal_cat           0
dtype: int64

In [24]:
# drop the nans
data.dropna(inplace=True)

In [25]:
data.isnull().sum()

title           0
salary          0
location        0
advertiser      0
industry        0
description     0
final_salary    0
sal_cat         0
dtype: int64

In [35]:
# let's pull out the keywords from title.

total_words = [title for title in data['title']]

vectorizer = CountVectorizer(analyzer = "word",ngram_range=(1,2),stop_words = 'english') 
train_data_features = vectorizer.fit_transform(total_words)

title_keys = pd.DataFrame(vectorizer.transform(total_words).todense(), columns=vectorizer.get_feature_names())

title_keys.sum().nlargest(35)

data                225
manager             169
analyst             137
senior               87
officer              78
business             75
engineer             70
support              61
sales                55
customer             48
administration       47
administrator        44
marketing            43
analytics            42
developer            41
scientist            40
data scientist       38
assistant            36
coordinator          36
digital              36
consultant           34
account              32
data analyst         30
development          28
executive            28
big                  27
big data             27
service              27
sydney               27
services             26
account manager      23
customer service     22
specialist           22
entry                21
insights             21
dtype: int64

In [36]:
title_keys[['data scientist']]

,data scientist
0,0
1,1
2,0
3,0
4,0
5,0
6,1
7,0
8,1
9,0


In [37]:
# And now we have our target! Now to pull out the keywords from description the same way we did in Part 1
stop = stopwords.words('english')
stop += ['xa0', 'span', 'xc2', 'xe3']
stop.append("need")
stop.append("in")
stop.append("re")
stop.append("are")
stop.append("for")
stop.append("have")
stop.append("like")
stop.append("be")
stop.append("see")
stop.append("do")
stop.append("trade")
stop.append("u2026")
stop.append("http")
stop.append("mrzlgv")
stop.append("bit")
stop.append("ly")
stop.append("div")
stop.append("class")
stop.append("forum")
stop.append("item")
stop.append("shar")
stop.append("profile")
stop.append("es")
stop += ['li', 'ul', 'br', ' ul', 'strong', 'xe2','x80','font','size','family','xa2', 'sans','serif','style','text',
         'margin','align','0cm','line height','10', '0pt', '11pt', 'calibri', 'com', 'au','height', 'normal', 'line',
        'em', 'contact', 'zealand', 'button', 'color','black','end','join','click','videoembed','flashcontent','id',
        'note','cover','letter','day','may','2017','please','apply']

In [38]:
total_desc_words = [desc for desc in data['description']]

vectorizer = CountVectorizer(analyzer = "word",ngram_range=(2,3),max_features=50,stop_words = stop) 
train_data_features = vectorizer.fit_transform(total_desc_words)

desc_keys = pd.DataFrame(vectorizer.transform(total_desc_words).todense(), columns=vectorizer.get_feature_names())

desc_keys.columns

Index(['ability work', 'attention detail', 'big data', 'business intelligence',
       'communication skills', 'confidential discussion', 'customer service',
       'data analysis', 'data analytics', 'data driven', 'data entry',
       'data management', 'data science', 'excellent communication',
       'excellent communication skills', 'exciting opportunity',
       'experience using', 'experience working', 'fast paced',
       'financial services', 'full time', 'high level', 'high quality',
       'highly regarded', 'ideal candidate', 'internal external',
       'interpersonal skills', 'key responsibilities', 'machine learning',
       'management skills', 'microsoft office', 'minimum years',
       'previous experience', 'problem solving', 'project management',
       'responsibilities include', 'skills ability', 'skills experience',
       'successful candidate', 'sydney cbd', 'team environment',
       'tertiary qualifications', 'time management', 'track record',
       'verbal co

In [39]:
# let's process industry!
total_words = [title for title in data['industry']]

vectorizer = CountVectorizer(analyzer = "word",ngram_range=(1,2),stop_words = 'english') 
train_data_features = vectorizer.fit_transform(total_words)

industry_keys = pd.DataFrame(vectorizer.transform(total_words).todense(), columns=vectorizer.get_feature_names())

industry_keys.sum().nlargest(35)

technology                   306
communication                274
communication technology     274
information                  274
information communication    274
administration               115
administration office        115
office                       115
office support               115
support                      115
services                      85
communications                70
marketing                     70
marketing communications      70
sales                         60
accounting                    59
banking                       52
banking financial             52
financial                     52
financial services            52
science                       32
science technology            32
defence                       28
government                    28
government defence            28
trades                        28
trades services               28
advertising                   21
advertising arts              21
arts                          21
arts media

In [40]:
# looking at our dataframe again
data

,title,salary,location,advertiser,industry,description,final_salary,sal_cat
0,Project Coordinator - Data entry,b'Up to $42ph plus Super',Sydney,Preacta Recruitment,Administration & Office Support,"b'<div class=""videoembed"" id=""FlashContent""></...",82992,high
1,Data Engineer / Data Scientist,b'Start up culture',Sydney,Motion Recruitment,Information & Communication Technology,"b""\n<p>Australian start up success story requi...",0,0.0
2,Data Analyst,0,Sydney,Westpac Group,Banking & Financial Services,b'<span>The Customer Data Analyst will support...,0,0.0
3,Media Data Analyst,0,Sydney,Big Wave Digital,Science & Technology,b'\n<div>\n<p>Are you looking to join massive ...,0,0.0
6,Data Steward - SAP / MDM,"b'$70,000 to $80,000 + super'",Sydney,Perigon Group Pty Limited,Information & Communication Technology,"b""Data Steward - SAP / MDM<br/>\n<br/>\r\nWe a...",75000,good
7,Data Engineer,0,Sydney,IAG,Information & Communication Technology,"b""\n<ul>\n<li>One of Australia's leading gener...",0,0.0
8,Data Scientist,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0.0
9,Data Analyst,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0.0
10,Data Scientist,0,Sydney,Caltex Australia,Science & Technology,"b""\n<p>At Caltex, we\xe2\x80\x99re with you al...",0,0.0
12,Data & Market Analyst,0,Sydney,Origin,Consulting & Strategy,"b'<div class=""videoembed"" id=""FlashContent""></...",0,0.0


In [41]:
# don't need title, don't need salary, don't need description, don't need location
data.drop('title',axis=1,inplace=True)
data.drop('description',axis=1,inplace=True)
data.drop('salary',axis=1, inplace=True)
data.drop('location',axis=1, inplace=True)
data.drop('industry',axis=1, inplace=True)

In [42]:
# oops. realised that sal_cat needs to be a numerical categorical column. 
# original code
#def salary_categoriser(salary):
#    salary = float(salary)
#    if salary == 0:
#        pass
#    elif salary < 80000:
#        salary = 'good'
#    elif salary >= 80000:
#        salary = 'high'
#    return salary

def salary_num(salary):
    salary = str(salary)
    if salary == 'good':
        salary = 1
    elif salary == 'high':
        salary = 2
    return salary

In [43]:
data['cat_salary'] = [salary_num(sal) for sal in data['sal_cat']]

In [44]:
data

,advertiser,final_salary,sal_cat,cat_salary
0,Preacta Recruitment,82992,high,2
1,Motion Recruitment,0,0.0,0.0
2,Westpac Group,0,0.0,0.0
3,Big Wave Digital,0,0.0,0.0
6,Perigon Group Pty Limited,75000,good,1
7,IAG,0,0.0,0.0
8,NRMA Motoring &amp; Services,0,0.0,0.0
9,NRMA Motoring &amp; Services,0,0.0,0.0
10,Caltex Australia,0,0.0,0.0
12,Origin,0,0.0,0.0


In [45]:
data.drop('advertiser',axis=1, inplace=True)
data.drop('sal_cat',axis=1, inplace=True)

In [46]:
data = pd.concat([data, industry_keys, desc_keys], axis=1)
data.shape

(1061, 141)

In [48]:
# and now we add our target variable
data['Scientist'] = title_keys[['data scientist']]

In [50]:
data.isnull().sum()

final_salary                    130
cat_salary                      130
accounting                      130
administration                  130
administration office           130
advertising                     130
advertising arts                130
architecture                    130
arts                            130
arts media                      130
banking                         130
banking financial               130
centre                          130
centre customer                 130
communication                   130
communication technology        130
communications                  130
community                       130
community services              130
construction                    130
consulting                      130
consulting strategy             130
consumer                        130
consumer products               130
customer                        130
customer service                130
defence                         130
design                      

In [51]:
# okay, let's drop all the nans in preparation to run logreg
data.dropna(inplace=True)

In [52]:
data.shape

(801, 142)

In [53]:
data.isnull().sum()

final_salary                    0
cat_salary                      0
accounting                      0
administration                  0
administration office           0
advertising                     0
advertising arts                0
architecture                    0
arts                            0
arts media                      0
banking                         0
banking financial               0
centre                          0
centre customer                 0
communication                   0
communication technology        0
communications                  0
community                       0
community services              0
construction                    0
consulting                      0
consulting strategy             0
consumer                        0
consumer products               0
customer                        0
customer service                0
defence                         0
design                          0
design architecture             0
development   

## Logistic Regression

In [87]:
predictors = [col for col in data.columns if col not in ['Scientist']]
y = data.Scientist
X = data[predictors]

In [88]:
ss = StandardScaler()
Xs = ss.fit_transform(X)

In [89]:
print('y: ', y.shape)
print('X: ', X.shape)
print('Xs: ', Xs.shape)

y:  (801,)
X:  (801, 145)
Xs:  (801, 145)


In [90]:
# baseline accuracy
y.value_counts()/y.count()

0.0    0.957553
1.0    0.042447
Name: Scientist, dtype: float64

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=8)

In [92]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

1.0

In [93]:
scores = cross_val_score(lr, X_test, y_test, cv=10)
predictions = cross_val_predict(lr, X_test, y_test, cv=10)

/Users/lohyenwei/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/Users/lohyenwei/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


In [94]:
scores.mean()

0.98235294117647065

In [95]:
# okay! looks like logistic regression does slightly better than baseline.

## Decision Tree Classifier

In [96]:
dtc1 = DecisionTreeClassifier(max_depth=1)
dtc2 = DecisionTreeClassifier(max_depth=2)
dtc3 = DecisionTreeClassifier(max_depth=3)
dtcN = DecisionTreeClassifier(max_depth=None)

In [97]:
dtc1.fit(Xs, y)
dtc2.fit(Xs, y)
dtc3.fit(Xs, y)
dtcN.fit(Xs, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [98]:
dtc1_scores = cross_val_score(dtc1, Xs, y, cv=4)
dtc2_scores = cross_val_score(dtc2, Xs, y, cv=4)
dtc3_scores = cross_val_score(dtc3, Xs, y, cv=4)
dtcN_scores = cross_val_score(dtcN, Xs, y, cv=4)

print(dtc1_scores, np.mean(dtc1_scores))
print(dtc2_scores, np.mean(dtc2_scores))
print(dtc3_scores, np.mean(dtc3_scores))
print(dtcN_scores, np.mean(dtcN_scores))

[ 0.89552239  0.9800995   0.98        0.9798995 ] 0.958880347009
[ 0.90049751  0.97512438  0.975       0.97487437] 0.956374065602
[ 0.90049751  0.97512438  0.975       0.99497487] 0.96139919123
[ 0.89054726  0.960199    0.975       0.98994975] 0.95392400435


In [ ]:
# okay, looks like decision tree classifiers do better than baseline at a depth of 3, but logreg does better than all

In [100]:
print(X.dtypes)
print(y.dtypes)

final_salary                    float64
cat_salary                       object
accounting                      float64
administration                  float64
administration office           float64
advertising                     float64
advertising arts                float64
architecture                    float64
arts                            float64
arts media                      float64
banking                         float64
banking financial               float64
centre                          float64
centre customer                 float64
communication                   float64
communication technology        float64
communications                  float64
community                       float64
community services              float64
construction                    float64
consulting                      float64
consulting strategy             float64
consumer                        float64
consumer products               float64
customer                        float64


In [101]:
for_corr = X.drop('cat_salary',axis=1)

In [103]:
science_corr=for_corr.corrwith(y)
print(science_corr.sort_values(ascending=False))

machine learning                0.659274
data science                    0.612856
science technology              0.439326
science                         0.439326
data analytics                  0.206560
financial services              0.189464
financial services              0.189464
data analysis                   0.180090
technology                      0.167106
data driven                     0.162431
big data                        0.151192
consulting strategy             0.140856
consulting                      0.140856
strategy                        0.140856
insurance                       0.103403
insurance superannuation        0.103403
superannuation                  0.103403
skills experience               0.094705
experience using                0.093423
work closely                    0.046221
highly regarded                 0.041112
years experience                0.031533
written verbal communication    0.028826
internal external               0.027840
verbal communica

### Okay, according to above, job postings that contain the keywords below are most likely to be for data scientists
#### machine learning         (0.659274)
#### data science                 (0.612856)
#### science technology     (0.439326)
#### science                         (0.439326)
